In [13]:
import pickle
import pandas as pd
import numpy as np
from fastdtw import fastdtw

In [14]:

with open('./data_del_some0_np.pickle', 'rb') as file:
    data_inflow = pickle.load(file)

In [15]:
data_inflow.shape

(4248, 308, 1)

In [16]:
data_inflow[:73, 0, 0]

array([  0,   0,  20,  20,  36,  24,  55,  52,  53,  46,  43,  59,  70,
        70,  66,  87,  74,  54,  68,  72,  61, 108,  79,  81,  95,  89,
        87, 105,  85,  84,  67,  71,  89,  79,  79,  45,  56,  53,  70,
        63,  59,  95,  63,  45,  57,  65,  99,  80,  67, 101,  37,  94,
       122,  84,  61,  54,  67,  69,  66,  42,  62,  48,  52,  49,  57,
        67,  48,  34,  52,  43,  33,  11,   0], dtype=int32)

In [17]:
data = data_inflow[:2969, :, 0]

In [18]:
data_t = data.T

In [19]:
data_t.shape

(308, 2969)

In [20]:


def calculate_dtw_distance(seq_a, seq_b):
    distance, path = fastdtw(seq_a, seq_b)
    return distance

In [21]:
from tqdm import tqdm

dtw_distances = np.zeros((308, 308))
for i in tqdm(range(308), desc="Processing", unit="item"):
    for j in range(i+1, 308):
        dtw_distances[i, j] = calculate_dtw_distance(data_t[i], data_t[j])
        dtw_distances[j, i] = dtw_distances[i, j]

Processing: 100%|██████████| 308/308 [1:19:09<00:00, 15.42s/item]


In [22]:
dtw_distances

array([[     0., 188267., 119476., ..., 173782., 271687., 267951.],
       [188267.,      0., 166538., ..., 100199., 107734., 107296.],
       [119476., 166538.,      0., ..., 133017., 227464., 203037.],
       ...,
       [173782., 100199., 133017., ...,      0., 121611., 115904.],
       [271687., 107734., 227464., ..., 121611.,      0.,  31753.],
       [267951., 107296., 203037., ..., 115904.,  31753.,      0.]])

In [23]:
npy_file_path = 'urban_bus_io_dtw.npy'

In [24]:
np.save(npy_file_path, dtw_distances)

In [25]:
my_list = dtw_distances[0]

In [31]:
data_dtw = np.load('./urban_bus_io_dtw.npy')

In [32]:
data_dtw.shape

(308, 308)

In [33]:
data = data_dtw

In [34]:
origin_id = [([i]*len(data[:, 0]))for i in range(len(data[:, 0]))]
destination_id = [[j for j in range(0, len(data[:, 0]))]
                  for i in range(len(data[:, 0]))]
connection = [list(data[i, :])for i in range(len(data[:, 0]))]

In [35]:
origin_id = [i for item in origin_id for i in item]
destination_id = [i for item in destination_id for i in item]
connection = [i for item in connection for i in item]

In [36]:
rel_id = [i for i in range(len(origin_id))]
type = ['geo']*len(origin_id)

In [37]:
d = {'rel_id': rel_id, 'type': type, 'origin_id': origin_id,
     'destination_id': destination_id, 'connection': connection}
data = pd.DataFrame(data=d)

In [38]:
data.to_csv('./URBAN_BUS_ROUTE.rel', index=False)